# Utils
## Base functions

In [ ]:
def identity(x):
    """"
    Identity method return parameter
    """
    return x

## Image utils

Please install : py-opencv

### Read & display images

In [ ]:
def assert_img_path(img_path, debug=False):
    if (debug):
        print('Loading image at ' + img_path)
    if img_path.endswith('.DS_Store'):
        raise Exception('Attention, wrong image path=' + img_path)        

In [ ]:
from imageio import imread

def img_read(img_path, debug=False):
    """"
    Read an image in RGB using imageio library
    """
    assert_img_path(img_path, debug)

    return imread(img_path)

In [ ]:
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_image_display/py_image_display.html
import numpy as np
import cv2

def img_read_cv(img_path, color_mode=cv2.IMREAD_COLOR, debug=False):
    """
    Read an image in BGR using cv2
    color_mode : cv2.IMREAD_UNCHANGED (-1) | cv2.IMREAD_GRAYSCALE (0) | cv2.IMREAD_COLOR (default : 1)
    """
    assert_img_path(img_path, debug)
    
    img = cv2.imread(img_path, color_mode)
    return img

# https://www.ccoderun.ca/programming/doxygen/opencv/group__imgproc__color__conversions.html
def img_convert_color(img, color_mode=cv2.COLOR_BGR2RGB):
    """
    Convert an image from BGR to RGB
    """
    img = cv2.cvtColor(img, color_mode)
    return img

In [ ]:
# https://matplotlib.org/tutorials/introductory/images.html
# https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.imshow.html
# https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.imshow
from matplotlib import pyplot as plt

def img_show(img, func=identity):
    """
    Display below cell an image using matplotlib
    """
    plt.imshow(func(img))
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

# https://matplotlib.org/3.1.1/tutorials/colors/colormaps.html
def img_show_using_color(img, color_map='gray'):
    """
    Display an image using the color map using matplotlib
    NOTE : IT DOESN'T CHANGE the color map, it INTERPRETS IT
    """
    plt.imshow(img, cmap=color_map)
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

def img_show_axis(img, axis, title=None, func=identity):
    axis.imshow(func(img))
    axis.axis("off")
    if title is None:
        axis.set_title(img)
    else:
        axis.set_title(title)

def img_show_axis_values(img_path, values, figsize=(15, 5)):
    fig, axs = plt.subplots(1, 2, figsize=figsize)

    # Box 1 - Image
    img = img_read_cv(img_path)
    img = img_convert_color(img)
    img_show_axis(img=img, title=img_path, axis=axs[0])
    
    # Box 2 - Values
    values.plot(kind="barh", figsize=figsize, ax=axs[1])
    
    plt.tight_layout()
    plt.show()

### Modify images

In [ ]:
from skimage.transform import resize

def resize_img(img, image_size=160):
    """
    Resize an image based on a standard size
    """
    return resize(img, (image_size, image_size), mode='reflect')

## Object detection

* More info for classifer at https://www.superdatascience.com/blogs/opencv-face-detection
* More info for OpenCV modules at https://docs.opencv.org/3.4.1/modules.html

In [ ]:
import cv2

lbpcascade_dir = '../cascades/lbpcascades/'

haarcascade_dir = '../cascades/haarcascades/'
cascade_frontalface_alt2 = cv2.CascadeClassifier(haarcascade_dir + 'haarcascade_frontalface_alt2.xml')

# https://stackoverflow.com/questions/36218385/parameters-of-detectmultiscale-in-opencv-using-python
def detect_object(img, scale_factor=1.1, min_neighbors=5, coordinate_mapper=None, cascade=cascade_frontalface_alt2
                 , debug=False):
    """
    Detect object specified with cascade
    """
    # https://docs.opencv.org/3.4.1/d1/de5/classcv_1_1CascadeClassifier.html#aaf8181cb63968136476ec4204ffca498
    faces = cascade.detectMultiScale(img,
                                     scaleFactor=scale_factor,
                                     minNeighbors=min_neighbors)
    if (debug):
        print('Item found: ', len(faces))
        
    if (coordinate_mapper == None):
        return faces
    
    face_collector = []
    for (x, y, w, h) in faces:
        face = coordinate_mapper(x, y, w, h)
        face_collector.append(face)
        
    return face_collector

In [ ]:
def extract_object_mapper_margin(img, margin=10, debug=False):
    """
    Currying a object extractor from an input img and using (x, y, w, h)
    """
    def extract_object_mapper(x, y, w, h):
        if (debug):
            print(x, y, w, h)
        cropped = img[y-margin//2:y+h+margin//2,
                      x-margin//2:x+w+margin//2, :]
        return cropped
    return extract_object_mapper

def highlight_object_mapper_color(img, color=(0, 255, 0), debug=False):
    """
    Currying a object highlighter from an input img and using (x, y, w, h)
    """
    def highlight_object_mapper(x, y, w, h):
        if (debug):
            print(x, y, w, h)
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 3)
    return highlight_object_mapper        